In [12]:
import rioxarray
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd

import dask

In [37]:
from dask.distributed import Client
client = Client()

/Users/marthamorrissey/mambaforge/envs/redwood-dev/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49751 instead
  warnings.warn(


In [50]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:49751/status,
Dashboard: http://127.0.0.1:49751/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49752,Workers: 4
Dashboard: http://127.0.0.1:49751/status,Total threads: 8
Started: 4 hours ago,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:49765,Total threads: 2
Dashboard: http://127.0.0.1:49768/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:49755,


In [14]:
dset = rioxarray.open_rasterio('/Users/marthamorrissey/Downloads/simple_dist_stack_Southern_Rockies.tif',
                               band_as_variable=True)

In [47]:
dset

<xarray.Dataset>
Dimensions:      (x: 16339, y: 28057)
Coordinates:
  * x            (x) float64 -1.183e+06 -1.183e+06 ... -6.932e+05 -6.932e+05
  * y            (y) float64 2.243e+06 2.243e+06 ... 1.402e+06 1.402e+06
    spatial_ref  int64 0
Data variables: (12/22)
    band_1       (y, x) uint8 ...
    band_2       (y, x) uint8 ...
    band_3       (y, x) uint8 ...
    band_4       (y, x) uint8 ...
    band_5       (y, x) uint8 ...
    band_6       (y, x) uint8 ...
    ...           ...
    band_17      (y, x) uint8 ...
    band_18      (y, x) uint8 ...
    band_19      (y, x) uint8 ...
    band_20      (y, x) uint8 ...
    band_21      (y, x) uint8 ...
    band_22      (y, x) uint8 ...
Attributes:
    AREA_OR_POINT:  Area

In [15]:
co_boundary = gpd.read_file('/Users/marthamorrissey/Downloads/Colorado_State_Boundary.geojson')

In [16]:
co_boundary_reproject = co_boundary.to_crs(dset.rio.crs)

In [17]:
dset_clipped = dset.rio.clip_box(*co_boundary_reproject.total_bounds)

In [6]:
dset_clipped.nbytes / 1e9

5.617235344

In [6]:
np.unique(dset['band_17'])

array([  0,   1,   2, 255], dtype=uint8)

In [18]:
years = range(1999, 2021)
time_coords = pd.to_datetime([f'{year}-01-01' for year in years])  # Convert years to datetime objects

In [19]:
len(time_coords)

22

In [20]:
time_lst = []

for i, v in enumerate(dset.data_vars):
    dset_tmp = dset[v].to_dataset().assign_coords(time=time_coords[i])
    dset_tmp = dset_tmp.rename({list(dset_tmp.data_vars)[0] : 'dist'})
    dset_tmp = dset_tmp.chunk(chunks="auto")
    time_lst.append(dset_tmp)

In [48]:
time_lst = []

for i, v in enumerate(dset.data_vars):
    dset_tmp = dset[v].rename({'band': 'time'})
    dset_tmp['time'] = time_coords[i]
    time_lst.append(dset_tmp)

combined_data = xr.concat(time_lst, dim='time')

ValueError: cannot rename 'band' because it is not a variable or dimension in this dataset

In [22]:
dset_format = xr.concat(time_lst, dim='time')

In [46]:
dset_format

<xarray.Dataset>
Dimensions:      (x: 16339, y: 28057, time: 22)
Coordinates:
  * x            (x) float64 -1.183e+06 -1.183e+06 ... -6.932e+05 -6.932e+05
  * y            (y) float64 2.243e+06 2.243e+06 ... 1.402e+06 1.402e+06
    spatial_ref  int64 0
  * time         (time) datetime64[ns] 1999-01-01 2000-01-01 ... 2020-01-01
Data variables:
    dist         (time, y, x) uint8 dask.array<chunksize=(1, 11585, 11585), meta=np.ndarray>

In [25]:
dset_format['dist'][0]

<xarray.DataArray 'dist' (y: 28057, x: 16339)>
dask.array<getitem, shape=(28057, 16339), dtype=uint8, chunksize=(11585, 11585), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 -1.183e+06 -1.183e+06 ... -6.932e+05 -6.932e+05
  * y            (y) float64 2.243e+06 2.243e+06 ... 1.402e+06 1.402e+06
    spatial_ref  int64 0
    time         datetime64[ns] 1999-01-01
Attributes:
    STATISTICS_MAXIMUM:  1
    STATISTICS_MEAN:     -9999
    STATISTICS_MINIMUM:  0
    STATISTICS_STDDEV:   -9999
    _FillValue:          255
    scale_factor:        1.0
    add_offset:          0.0
    long_name:           forest-disturbance1999

In [42]:
# total count of each disturbance type per pixel --> write out to single band raster if possible 

# fire values 1 and 4 

# insect values 2 and 5 

fire_counts = dset_format.where((dset_format['dist'] == 1) | (dset_format['dist'] == 4)).count(dim='time').astype('uint8')


In [30]:
defaults = {
        "driver": "GTiff",
        "interleave": "pixel",
        "tiled": True,
        "blockxsize": 512,
        "blockysize": 512,
        "compress": "DEFLATE",
    }


fire_counts.rio.to_raster('/Users/marthamorrissey/Downloads/fire_counts.tif', **defaults)

2024-03-13 17:28:05,326 - distributed.scheduler - ERROR - Task ('sum-partial-b9b2c6d5ecb018fedc53eaa943dbef36', 2, 0, 0) has 4.00 GiB worth of input dependencies, but worker tcp://127.0.0.1:64652 has memory_limit set to 2.00 GiB.


MemoryError: Task ('sum-partial-b9b2c6d5ecb018fedc53eaa943dbef36', 2, 0, 0) has 4.00 GiB worth of input dependencies, but worker tcp://127.0.0.1:64652 has memory_limit set to 2.00 GiB.

In [31]:
fire_counts

<xarray.Dataset>
Dimensions:      (y: 28057, x: 16339)
Coordinates:
  * x            (x) float64 -1.183e+06 -1.183e+06 ... -6.932e+05 -6.932e+05
  * y            (y) float64 2.243e+06 2.243e+06 ... 1.402e+06 1.402e+06
    spatial_ref  int64 0
Data variables:
    dist         (y, x) int64 dask.array<chunksize=(11585, 11585), meta=np.ndarray>

In [ ]:
insect_counts = dset_format.where((dset_format['dist'] == 2) | (dset_format['dist'] == 5)).count(dim='time').astype('uint8')

In [33]:
southern_rockies = gpd.read_file('/Users/marthamorrissey/Downloads/southern_rockies_eco.geojson')

In [38]:
southern_rockies_r = southern_rockies.to_crs(dset.rio.crs)

In [36]:
fire_counts.rio.crs

CRS.from_epsg(5070)

In [43]:
fire_counts_clip = fire_counts.rio.clip(southern_rockies_r.geometry.values)

In [44]:
fire_counts

<xarray.Dataset>
Dimensions:      (y: 28057, x: 16339)
Coordinates:
  * x            (x) float64 -1.183e+06 -1.183e+06 ... -6.932e+05 -6.932e+05
  * y            (y) float64 2.243e+06 2.243e+06 ... 1.402e+06 1.402e+06
    spatial_ref  int64 0
Data variables:
    dist         (y, x) uint8 dask.array<chunksize=(11585, 11585), meta=np.ndarray>

In [45]:
fire_counts.rio.to_raster('/Users/marthamorrissey/Downloads/fire_counts_southern_rockies_clip.tif', **defaults)

2024-03-13 17:47:52,414 - distributed.scheduler - ERROR - Task ('sum-partial-b9b2c6d5ecb018fedc53eaa943dbef36', 2, 0, 0) has 4.00 GiB worth of input dependencies, but worker tcp://127.0.0.1:49764 has memory_limit set to 2.00 GiB.


MemoryError: Task ('sum-partial-b9b2c6d5ecb018fedc53eaa943dbef36', 2, 0, 0) has 4.00 GiB worth of input dependencies, but worker tcp://127.0.0.1:49764 has memory_limit set to 2.00 GiB.

2024-03-13 19:40:40,187 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 2 memory: 26 MB fds: 30>>
Traceback (most recent call last):
  File "/Users/marthamorrissey/mambaforge/envs/redwood-dev/lib/python3.10/site-packages/tornado/ioloop.py", line 919, in _run
    val = self.callback()
  File "/Users/marthamorrissey/mambaforge/envs/redwood-dev/lib/python3.10/site-packages/distributed/system_monitor.py", line 168, in update
    net_ioc = psutil.net_io_counters()
  File "/Users/marthamorrissey/mambaforge/envs/redwood-dev/lib/python3.10/site-packages/psutil/__init__.py", line 2126, in net_io_counters
    rawdict = _psplatform.net_io_counters()
OSError: [Errno 12] Cannot allocate memory


In [49]:
## take 2

time_lst = []

# Assuming you have a list of time coordinates called `time_coords`
for i, v in enumerate(dset.data_vars):
    dset_tmp = dset[v].expand_dims({'time': [time_coords[i]]})
    time_lst.append(dset_tmp)

combined_data = xr.concat(time_lst, dim='time')